In [1]:
import json
import logging
import os
import sys

from dotenv import load_dotenv
from loguru import logger
import numpy as np
import pandas as pd
import s3fs
import sqlalchemy
from sqlalchemy import text

import geopandas as gpd
from shapely.geometry import LineString

sys.path.append('..')


import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
from dash.exceptions import PreventUpdate
from loguru import logger
import io
import pandas as pd
import datetime as dt
import calendar
import numpy as np

%load_ext autoreload
%autoreload 2
pd.set_option('display.max_columns', None)

dataset_name = 'poc3'

In [2]:
df = pd.read_csv(f'../data/{dataset_name}.csv', 
                 low_memory=False,
                 parse_dates=['Pickup date'], 
                 dtype={'DC zip': str, 'Zip': str}).sort_values(by='Pickup date')
df = df.dropna(subset = ['Distance (km)'])

In [3]:
df

,Pickup date,Client no,DC name,Client name,Address,Zip,City,Country code,Carrier name,Sender weight (kg),Volume (m3),Nb of packages,Shipment id,DC code,Shipping type,Country,DC Name,DC address,DC city,DC zip,DC country,DC country code,Shipper longitude,Shipper latitude,Receiver longitude,Receiver latitude,Distance (km),Co2 emissions,Co2 diesel,Co2 BEV
0,2022-01-02 00:00:00,37580258-230,RDC Örebro,AHLSELL AB / GELIA,INDUSTRIVÄGEN,46784,GRÄSTORP,SE,DHL Sverige (SE),73.000,611.9490,1.0,9662065359,DK01,Road,Sweden,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,12.080903,57.573191,295.740,2158.902000,1791.888660,40.155577
308,2022-01-02 00:00:00,MULTIPLE,RDC Örebro,ONNINEN AS - D + 3 HOVEDLAGER,HØGSLUNDVEIEN 49 PORT 41,2020,SKEDSMOKORSET,NO,NTG Continent (DK),44.784,710.0310,1.0,"4,01707E+19",DK01,Road,Norway,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,11.051065,60.008642,317.272,1420.870925,1179.322868,26.428199
307,2022-01-02 00:00:00,VKO 4 2022,RDC Örebro,KESKUSMAA OY,Teletie 6 C,96100,NaN,FI,Posti Oy (FI),55.099,82.8138,6.0,JJFI61766000000328055,DK01,Road,Finland,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,25.654182,66.496605,1220.274,6723.587713,5580.577801,125.058731
306,2022-01-02 00:00:00,418037,RDC Örebro,HÖRBERG PETERSSON TRONIC AB,MÅSVÄGEN 42,22733,LUND,SE,DHL Sverige (SE),0.678,0.9840,1.0,9661815747,DK01,Road,Sweden,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,13.177360,55.712590,488.978,33.152708,27.516748,0.616640
305,2022-01-02 00:00:00,382,RDC Örebro,MORO COMPONENTI SRL,VIA PIOVEGA 15/A,35010,NaN,IT,DSV Sverige (SE),484.000,1721.0232,3.0,5990891003,DK01,Road,Italy,RDC Oerebro,Lastgatan 2,Örebro,70227,Sweden,SE,15.124890,59.243470,11.953840,45.539360,1974.308,95556.507200,79311.900976,1777.351034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491859,2022-12-30 18:02:00,1500077276,DC Evreux,SONEPAR FR DIST ST VULB ST 109,345 ALLEE DE CUREBOURSE,01150,SAINT VULBAS,FR,TRANSPORTS BREGER,16557.854,118.3490,NaN,CV01631120,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,5.282961,45.855417,577.005,955396.454727,792979.057423,76431.716378
491862,2022-12-30 18:07:00,1000071197,DC Evreux,SOREEL SAS,18 RUE DE LA GATINE,49300,CHOLET,FR,TRANSPORTS BREGER,1040.999,7.3940,NaN,CV01631116,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,-0.920033,47.037756,336.152,34993.389585,29044.513355,2799.471167
491860,2022-12-30 18:07:00,1000065092,DC Evreux,EFA FRANCE SARL,3 RUE CLAUDE CHAPPE,37230,FONDETTES,FR,TRANSPORTS BREGER,3342.139,14.6210,NaN,CV01631116,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,0.624121,47.411070,270.988,90567.956333,75171.403757,7245.436507
491861,2022-12-30 18:07:00,1500010296,DC Evreux,STAC CDL STOCK,300 AV ANTOINE LAVOISIER,44590,DERVAL,FR,TRANSPORTS BREGER,1357.641,10.8360,NaN,CV01631116,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,-1.662835,47.644423,369.566,50173.795381,41644.250166,4013.903630


In [4]:
df.columns

Index(['Pickup date', 'Client no', 'DC name', 'Client name', 'Address', 'Zip',
       'City', 'Country code', 'Carrier name', 'Sender weight (kg)',
       'Volume (m3)', 'Nb of packages', 'Shipment id', 'DC code',
       'Shipping type', 'Country', 'DC Name', 'DC address', 'DC city',
       'DC zip', 'DC country', 'DC country code', 'Shipper longitude',
       'Shipper latitude', 'Receiver longitude', 'Receiver latitude',
       'Distance (km)', 'Co2 emissions', 'Co2 diesel', 'Co2 BEV'],
      dtype='object')

## Test functions for app

In [5]:
import pandas as pd
import calendar

In [6]:
operators = [['ge ', '>='],
             ['le ', '<='],
             ['lt ', '<'],
             ['gt ', '>'],
             ['ne ', '!='],
             ['eq ', '='],
             ['contains '],
             ['datestartswith ']]


def split_filter_part(filter_part):
    for operator_type in operators:
        for operator in operator_type:
            if operator in filter_part:
                name_part, value_part = filter_part.split(operator, 1)
                name = name_part[name_part.find('{') + 1: name_part.rfind('}')]

                value_part = value_part.strip()
                v0 = value_part[0]
                if (v0 == value_part[-1] and v0 in ("'", '"', '`')):
                    value = value_part[1: -1].replace('\\' + v0, v0)
                else:
                    try:
                        value = float(value_part)
                    except ValueError:
                        value = value_part

                # word operators need spaces after them in the filter string,
                # but we don't want these later
                return name, operator_type[0].strip(), value
    return [None] * 3

def apply_filters(df, filter):
    # filtering
    filtering_expressions = filter.split(' && ')

    for filter_part in filtering_expressions:
        col_name, operator, filter_value = split_filter_part(filter_part)

        if operator in ('eq', 'ne', 'lt', 'le', 'gt', 'ge'):
            # these operators match pandas series operator method names
            df = df.loc[getattr(df[col_name], operator)(filter_value)]
        elif operator == 'contains':
            dff = dff.loc[dff[col_name].str.contains(filter_value)]
        elif operator == 'datestartswith':
            # this is a simplification of the front-end filtering logic,
            # only works with complete fields in standard format
            dff = dff.loc[dff[col_name].str.startswith(filter_value)]
    return dff

def apply_month_shipper(dff, month, shipper): 
    # apply month filter
    if month is None:
        dff = dff
    else:
        dff = dff[dff['Pickup date'].dt.month == month]
    
    # apply shipper filter
    if shipper is None:
        dff = dff
    else:
        dff = dff[dff['DC name'] == shipper]
    return dff

In [7]:
def apply_selected_rows_indices(dff, rows, selected_rows_indices):
    if (selected_rows_indices is None) or (selected_rows_indices == []):
        # No receiver selected
        dff = dff
    else:
        # filter for selected receiver
        df_select = pd.DataFrame(rows)
        logger.info(df_select.columns)
        receiver = df_select['Client name'].values[selected_rows_indices]
        receiver_lat = df_select['Receiver latitude'].values[selected_rows_indices]
        receiver_lon = df_select['Receiver longitude'].values[selected_rows_indices]
        dff = dff[(dff['Client name'].isin(receiver))&(dff['Receiver latitude'].isin(receiver_lat))&dff['Receiver longitude'].isin(receiver_lon)]
    return dff

In [9]:
def group_by_client_shipment(dff):
    dff = dff.groupby(['Receiver longitude', 'Receiver latitude', "Client name", 'Shipment id']).agg(
        distance=("Distance (km)", 'first'),
        weight=('Sender weight (kg)', 'sum'),
        volume=("Volume (m3)", 'sum'),
        count=('Client name', 'count'),
        co2_diesel=("Co2 diesel", 'sum'),
        co2_bev=("Co2 BEV", 'sum')
        ).reset_index()
    dff = dff.rename(columns={'distance': 'Distance (km)',
                              'weight': 'Sender weight (kg)', 'volume': 'Volume (m3)',
                              'count': 'Number of shippings', 'co2_diesel': 'Co2 diesel', 'co2_bev': 'Co2 BEV'})
    return dff

def group_by_client(dff):
    dff = dff.groupby(['Receiver longitude', 'Receiver latitude', "Client name"]).agg(
        distance=("Distance (km)", 'first'),
        weight=('Sender weight (kg)', 'sum'),
        volume=("Volume (m3)", 'sum'),
        shipments=('Number of shippings', 'sum'),
        co2_diesel=("Co2 diesel", 'sum'),
        co2_bev=("Co2 BEV", 'sum')
        ).reset_index()
    dff = dff.rename(columns={'distance': 'Distance (km)',
                              'weight': 'Sender weight (kg)', 'volume': 'Volume (m3)',
                              'shipments': 'Number of shippings', 'co2_diesel': 'Co2 emissions (g)', 'co2_bev': 'Co2 BEV (g)'})
    return dff

In [10]:
def create_map(df, selected_rows_indices, shipper='all', shipper_lat=50, shipper_long=2):
    fig = go.Figure()
    lat_mean = pd.concat([df['Shipper latitude'],df['Receiver latitude']]).unique().mean()
    long_mean = pd.concat([df['Shipper longitude'],df['Receiver longitude']]).unique().mean()
    if shipper == 'all':
        # map with all clients colored by dc
        for i in df['DC name'].unique():
            df_plot = df[df['DC name']== i]
            fig.add_trace(go.Scattermapbox(
                lat=df_plot["Receiver latitude"], lon=df_plot["Receiver longitude"], name = i,
                customdata=df[["Client name", "Distance (km)", "Sender weight (kg)"]],
                hovertemplate='''<br>Client: %{customdata[0]}<br>Distance: %{customdata[1]:,.0f} km<br>Weight: %{customdata[2]:,.0f} kg<br><extra></extra>'''))
        fig.update_layout(title = f'Map with clients per DC')
    else:
        # map with clients per dc
        fig.add_trace(go.Scattermapbox(
            lat=df["Receiver latitude"], lon=df["Receiver longitude"], name = "Clients", marker={'color': '#009530',  'size': 8},
            customdata=df[["Client name", "Distance (km)", "Sender weight (kg)"]],
            hovertemplate='''<br>Client: %{customdata[0]}<br>Distance: %{customdata[1]:,.0f} km<br>Weight: %{customdata[2]:,.0f} kg<br><extra></extra>'''))
        fig.add_trace(go.Scattermapbox(lat=[shipper_lat], lon=[
                    shipper_long], name=f'{shipper}', marker={'color': '#B10043', 'size': 12}))
        fig.update_layout(title = f'Map with clients of {shipper}')
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 40}, mapbox={
                      'zoom': 4, 
                      "center": {'lat': lat_mean, 'lon': long_mean}})
    fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01))
    return fig

In [11]:
def prepare_data_for_bar_plot(dff, month, selected_rows_indices):
    dff['Pickup date'] = pd.to_datetime(dff['Pickup date'])

    if (selected_rows_indices is None) or (selected_rows_indices == []):
        dff['Pickup date'] = dff['Pickup date'].dt.date
        dff = dff.groupby(['Pickup date', 'Carrier name']).agg({
            'Sender weight (kg)': sum,
            'Volume (m3)': sum}).reset_index()
    else:
        dff = dff.groupby(['Pickup date', 'Shipment id', 'Carrier name']).agg({
            'Client name': lambda tdf: tdf.unique(),
            'Sender weight (kg)': sum,
            'Volume (m3)': sum}).reset_index()
        
    dff['Pickup date'] = pd.to_datetime(dff['Pickup date'])
    if month is not None:
        # prepare array that includes all days in that month
        year = 2022
        first, last = calendar.monthrange(year, month)
        dt_all = pd.date_range(
            start = dt.datetime(year, month, 1),
            end = dt.datetime(year, month, last),
            freq='D')
        time_period = f'{calendar.month_name[month]} '
    else:
        # prepare array that includes all days in time period of df
        time_period = ''
        dt_all = pd.date_range(
            start=dt.datetime(2022, 1, 1),
            end=dt.datetime(2022, 12, 31),
            freq='D')

    # prepare df with one row per day and set Sender weight by default to 0
    df_dt_all = pd.DataFrame(dt_all).rename(columns={0: "Key"})
    df_dt_all['Key'] = pd.to_datetime(df_dt_all['Key']) 

    # add all days to dataset
    dff['Key'] = pd.to_datetime(dff['Pickup date'].dt.date)
    df_dt_all = df_dt_all.merge(dff, on=['Key'], how='outer').fillna({'Sender weight (kg)': 0, 'Volume (m3)': 0})
    df_dt_all['Pickup date'].fillna(df_dt_all['Key'], inplace = True)
    df_dt_all = df_dt_all.sort_values(by='Pickup date')
    return df_dt_all



In [12]:
def prepare_data_bar_plot(dff, selected_rows_indices):
    dff['Pickup date'] = pd.to_datetime(dff['Pickup date'])

    if (selected_rows_indices is None) or (selected_rows_indices == []):
        dff['Pickup date'] = dff['Pickup date'].dt.date
        dff = dff.groupby(['Pickup date', 'Carrier name']).agg({
            'Sender weight (kg)': sum,
            'Volume (m3)': sum}).reset_index()
    else:
        dff = dff.groupby(['Pickup date', 'Shipment id', 'Carrier name']).agg({
            'Client name': lambda tdf: tdf.unique(),
            'Sender weight (kg)': sum,
            'Volume (m3)': sum}).reset_index()
        
    dff['Pickup date'] = pd.to_datetime(dff['Pickup date'])
    return dff

## Check data after prep

In [32]:
month = None
dff = df

selected_rows_indices = [1,2]
shipper = 'DC Evreux'

rows = dff[dff['DC name'] == shipper].reset_index()
if shipper == None:
    rows = dff.reset_index()

dff['Pickup date'] = pd.to_datetime(dff['Pickup date'])
dff = apply_month_shipper(dff, month, shipper)
#dff = dff[dff['Client name'] =='ETT ENERG.TRANSF.THER']
dff = apply_selected_rows_indices(dff, rows, selected_rows_indices)
dff['Pickup date'] = pd.to_datetime(dff['Pickup date'])
df_for_plot = prepare_data_bar_plot(dff, selected_rows_indices)


2023-03-15 16:37:50.482 | INFO     | __main__:apply_selected_rows_indices:8 - Index(['index', 'Pickup date', 'Client no', 'DC name', 'Client name',
       'Address', 'Zip', 'City', 'Country code', 'Carrier name',
       'Sender weight (kg)', 'Volume (m3)', 'Nb of packages', 'Shipment id',
       'DC code', 'Shipping type', 'Country', 'DC Name', 'DC address',
       'DC city', 'DC zip', 'DC country', 'DC country code',
       'Shipper longitude', 'Shipper latitude', 'Receiver longitude',
       'Receiver latitude', 'Distance (km)', 'Co2 emissions', 'Co2 diesel',
       'Co2 BEV'],
      dtype='object')


In [28]:
df_for_plot = prepare_data_for_bar_plot(dff, month, selected_rows_indices)

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_21832\827015732.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\SESA667490\AppData\Local\Temp\ipykernel_21832\827015732.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
dff

,Pickup date,Client no,DC name,Client name,Address,Zip,City,Country code,Carrier name,Sender weight (kg),Volume (m3),Nb of packages,Shipment id,DC code,Shipping type,Country,DC Name,DC address,DC city,DC zip,DC country,DC country code,Shipper longitude,Shipper latitude,Receiver longitude,Receiver latitude,Distance (km),Co2 emissions,Co2 diesel,Co2 BEV
1118,2022-01-03 07:58:00,1000072630,DC Evreux,ENEDIS,"126, RUE EMILE COMBES",33270,FLOIRAC,FR,SCHENKER FRANCE,234.000,1.440,NaN,CV01202364,FR45,49,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,-0.539142,44.832510,622.026,14555.408400,12080.988972,1164.432672
1295,2022-01-03 11:59:00,1000052805,DC Evreux,GROUPE SYSELEC SAS,AV DU 8 MAI 1945,81290,LABRUGUIERE,FR,TRANS SERVICE,73.000,0.432,NaN,CV01210772,FR45,49,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,2.254805,43.543628,747.433,5456.260900,4528.696547,436.500872
1302,2022-01-03 11:59:00,1000052805,DC Evreux,GROUPE SYSELEC SAS,AV DU 8 MAI 1945,81290,LABRUGUIERE,FR,TRANS SERVICE,43.092,0.231,NaN,CV01210425,FR45,49,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,2.254805,43.543628,747.433,3220.838284,2673.295775,257.667063
1297,2022-01-03 11:59:00,1000052805,DC Evreux,GROUPE SYSELEC SAS,AV DU 8 MAI 1945,81290,LABRUGUIERE,FR,TRANS SERVICE,64.201,0.199,NaN,CV01210589,FR45,49,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,2.254805,43.543628,747.433,4798.594603,3982.833521,383.887568
3797,2022-01-05 12:48:00,1000052805,DC Evreux,GROUPE SYSELEC SAS,AV DU 8 MAI 1945,81290,LABRUGUIERE,FR,TRANS SERVICE,115.578,0.412,NaN,CV01212233,FR45,49,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,2.254805,43.543628,747.433,8638.681127,7170.105336,691.094490
29266,2022-01-27 08:22:00,1000072630,DC Evreux,ENEDIS,"126, RUE EMILE COMBES",33270,FLOIRAC,FR,SCHENKER FRANCE,234.000,1.440,NaN,CV01235300,FR45,49,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,-0.539142,44.832510,622.026,14555.408400,12080.988972,1164.432672
33109,2022-01-31 12:22:00,1000052805,DC Evreux,GROUPE SYSELEC SAS,AV DU 8 MAI 1945,81290,LABRUGUIERE,FR,TRANS SERVICE,147.029,0.690,NaN,CV01241073,FR45,49,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,2.254805,43.543628,747.433,10989.432656,9121.229104,879.154612
36136,2022-02-02 11:27:00,1000052805,DC Evreux,GROUPE SYSELEC SAS,AV DU 8 MAI 1945,81290,LABRUGUIERE,FR,TRANS SERVICE,32.553,0.111,NaN,CV01244495,FR45,49,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,2.254805,43.543628,747.433,2433.118645,2019.488475,194.649492
41312,2022-02-07 07:28:00,1000072630,DC Evreux,ENEDIS,"126, RUE EMILE COMBES",33270,FLOIRAC,FR,XPO DISTRIBUTION FRANCE,702.000,4.320,NaN,CV01245273,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,-0.539142,44.832510,622.026,43666.225200,36242.966916,3493.298016
70190,2022-03-02 08:41:00,1000072630,DC Evreux,ENEDIS,"126, RUE EMILE COMBES",33270,FLOIRAC,FR,XPO DISTRIBUTION FRANCE,585.000,3.600,NaN,CV01272506,FR45,42,France,DC Evreux,Rue Roland Garos,Guichainville,27900,France,FR,1.179094,49.005514,-0.539142,44.832510,622.026,36388.521000,30202.472430,2911.081680


###  Client selected

In [14]:
def bar_plot_client(df, month):
    df['Pickup date'] = pd.to_datetime(df["Pickup date"])
    df['Pickup time'] = df["Pickup date"].dt.time
    df['Pickup date'] = df["Pickup date"].dt.date
    fig = go.Figure()
    for i in df['Carrier name'].unique():
        df_plot = df[df['Carrier name']==i]
        fig.add_trace(go.Bar(
            x=df_plot["Pickup date"],
            y=df_plot["Sender weight (kg)"],
            name = i,
            xperiodalignment="middle",
            customdata=df_plot[["Pickup time", "Sender weight (kg)", "Volume (m3)", 'Shipment id']],
            hovertemplate='<br>%{x}Time: %{customdata[0]}<br>Weight: %{customdata[1]:,.3f}<br>Volume: %{customdata[2]:,.3f}<br>%{customdata[3]}<br><extra></extra>'
        ))
    if month is not None:
        first, last = calendar.monthrange(2022, month)
        fig.update_layout(xaxis_range=[dt.datetime(2022, month, 1),
                                        dt.datetime(2022, month, last)])
        fig.update_xaxes(dtick=86400000.0)
    else:
        fig.update_layout(xaxis_range=[dt.datetime(2022, 1, 1),
                                        dt.datetime(2022, 12, 31)])
    fig.update_layout(barmode='relative')
    return fig

In [19]:
def bar_px_plot_client(df, month):
    df['Pickup date'] = pd.to_datetime(df["Pickup date"])
    df['Pickup time'] = df["Pickup date"].dt.time
    df['Pickup date'] = df["Pickup date"].dt.date
    fig = px.histogram(df, x='Pickup date', y="Sender weight (kg)", color = 'Carrier name', barmode="stack")
    
    if month is not None:
        fig.update_layout(xaxis_range=[dt.datetime(2022, month, 1),
                                        dt.datetime(2022, 1, 31)])
        fig.update_xaxes(dtick=86400000.0, tickformat="%d\n%b %Y")
        fig.update_traces(xbins_size=86400000.0)
    else:
        fig.update_layout(xaxis_range=[dt.datetime(2022, 1, 1),
                                        dt.datetime(2022, 12, 31)])
        fig.update_traces(xbins_size='M1')
        fig.update_xaxes(ticklabelmode="period", dtick="M1",tickformat="%b\n%Y")
    fig.update_layout(bargap=0.1)
    fig.update_traces(customdata=df[["Pickup time", "Sender weight (kg)", "Volume (m3)", 'Shipment id']],
            hovertemplate='<br>%{x}<br>Time: %{customdata[0]}<br>Weight: %{customdata[1]:,.2f} kg<br>Volume: %{customdata[2]:,.3f} m3<br>%{customdata[3]}<br><extra></extra>')
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        bordercolor="grey",
        borderwidth=1,
        x=1))
    return fig

In [20]:
bar_px_plot_client(df_for_plot, None)

KeyError: "['Shipment id'] not in index"

In [16]:
def plot(df, month):
    df['Pickup date'] = pd.to_datetime(df["Pickup date"])
    df['Pickup time'] = df["Pickup date"].dt.time
    fig = px.bar(df, x='Pickup date', y="Sender weight (kg)", barmode="stack")
    if month is not None:
        fig.update_xaxes(dtick=86400000.0, minor = {'ticks': 'inside', 'tick0': dt.datetime(2022, 1, 1)})
    else:
        fig.update_layout(xaxis_range=[dt.datetime(2022, 1, 1),
                                        dt.datetime(2022, 12, 31)])

    fig.update_layout(barmode='relative', marker_color='rgb(158,202,225)')
    fig.update_traces(customdata=df[["Pickup time", "Sender weight (kg)", "Volume (m3)", 'Shipment id']],
            hovertemplate='<br>%{x}<br>Time: %{customdata[0]}<br>Weight: %{customdata[1]:,.2f} kg<br>Volume: %{customdata[2]:,.3f} m3<br>%{customdata[3]}<br><extra></extra>')
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        bordercolor="grey",
        borderwidth=1,
        x=1))
    fig.show()

### No client selected

In [29]:
def bar_px_plot_no_client(df, month):
    if shipper is None:
        df = df.groupby(['Pickup date'])[['Sender weight (kg)','Volume (m3)']].sum().reset_index()
        fig = px.histogram(df, x='Pickup date', y="Sender weight (kg)", barmode="stack")
        title = f"Shipments for all DC's"
    else:
        fig = px.histogram(df, x='Pickup date', y="Sender weight (kg)", color = 'Carrier name', barmode="stack")
        title = f'Shipments for {shipper}'
    if month is not None:
        fig.update_layout(xaxis_range=[dt.datetime(2022, month, 1),
                                        dt.datetime(2022, 1, 31)])
        fig.update_xaxes(dtick=86400000.0, tickformat="%d\n%b %Y")
        fig.update_traces(xbins_size=86400000.0)
    else:
        fig.update_layout(xaxis_range=[dt.datetime(2022, 1, 1),
                                        dt.datetime(2022, 12, 31)])
        fig.update_traces(xbins_size='M1')
        fig.update_xaxes(ticklabelmode="period", dtick="M1",tickformat="%b\n%Y")
    fig.update_layout(bargap=0.1)
    fig.update_traces(customdata=df[["Sender weight (kg)", "Volume (m3)"]],
            hovertemplate='<br>%{x}<br>Weight: %{customdata[0]:,.2f} kg<br>Volume: %{customdata[1]:,.3f} m3<br><extra></extra>')
    fig.update_layout(title = title)
    return fig

In [294]:
def bar_plot_no_client(df, month):
    fig = go.Figure()
    for i in df['Carrier name'].unique():
        df_plot = df[df['Carrier name']==i]
        fig.add_trace(go.Bar(
            x=df_plot["Pickup date"],
            y=df_plot["Sender weight (kg)"],
            name = i,
            xperiodalignment="middle",
            customdata=df_plot[["Sender weight (kg)", "Volume (m3)"]],
            hovertemplate='<br>%{x}<br>Weight: %{customdata[0]:,.2f} kg<br>Volume: %{customdata[1]:,.3f} m3<br><extra></extra>'
        ))
        if month is not None:
            first, last = calendar.monthrange(2022, month)
            fig.update_layout(xaxis_range=[dt.datetime(2022, month, 1),
                                           dt.datetime(2022, month, last)])
            fig.update_xaxes(dtick=86400000.0)
        else:
            fig.update_layout(xaxis_range=[dt.datetime(2022, 1, 1),
                                           dt.datetime(2022, 12, 31)])   
    fig.update_layout(barmode='relative')
    
    return fig

In [31]:
bar_px_plot_no_client(df_for_plot, None).show()